# Lab Workspace Set Up
***

### Update the Databricks Python SDK
***

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("customer_name", "", "Customer Name")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

### Set Customer Parameters
***

In [0]:
customer_name = dbutils.widgets.get("customer_name")

In [0]:
print(f"""
      customer_name = {customer_name}
""")

### Set Up the Default Catalog
***

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS fhir_workshop;

In [0]:
%sql
GRANT USE CATALOG ON CATALOG fhir_workshop TO `account users`;
GRANT CREATE SCHEMA ON CATALOG fhir_workshop TO `account users`;

In [0]:
%sql
REVOKE BROWSE ON CATALOG fhir_workshop FROM `account users`; 

In [0]:
%sql
SHOW GRANTS ON CATALOG fhir_workshop;

### Create Synthea Schemas 
***

In [0]:
%sql
use catalog fhir_workshop;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS synthea;
GRANT USE SCHEMA ON SCHEMA synthea TO `account users`;
GRANT SELECT ON SCHEMA synthea TO `account users`;
GRANT EXECUTE ON SCHEMA synthea TO `account users`;
GRANT READ VOLUME ON SCHEMA synthea TO `account users`;
    
SHOW GRANTS ON SCHEMA synthea;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS synthea_65k;
GRANT USE SCHEMA ON SCHEMA synthea_65k TO `account users`;
GRANT SELECT ON SCHEMA synthea_65k TO `account users`;
GRANT EXECUTE ON SCHEMA synthea_65k TO `account users`;
GRANT READ VOLUME ON SCHEMA synthea_65k TO `account users`;
    
SHOW GRANTS ON SCHEMA synthea_65k;

### Set fhir_workshop as the default Catalog
***

In [0]:
from databricks.sdk.service.settings import DefaultNamespaceSetting, StringMessage
from databricks.sdk.errors.platform import ResourceConflict

In [0]:
try:
    w.settings.default_namespace.update(
        allow_missing=True
        ,setting=DefaultNamespaceSetting(
            namespace=StringMessage(
                value="fhir_workshop"
            )
        )
        ,field_mask="namespace"
    )
except Exception as e:
    print(f"Default namespace already set, update with serializedCT as etag: {e}")

In [0]:
dfn = w.settings.default_namespace.get()

if dfn.namespace.value != "fhir_workshop":
  w.settings.default_namespace.update(
        allow_missing=True
        ,setting=DefaultNamespaceSetting(
            namespace=StringMessage(
                value="fhir_workshop"
            )
            ,etag = dfn.etag
        )
        ,field_mask="namespace"
    )
  dfn = w.settings.default_namespace.get()

print(f"""
    Default namespace is: {dfn.namespace.value}
""")

_Note that all compute resources must be completely restarted to take affect._

### Create the SQL Warehouse for the Class 
***

In [0]:
from databricks.sdk.service.sql import Channel, ChannelName, WarehouseAccessControlRequest, WarehousePermissionLevel

In [0]:
warehouses = w.warehouses.list()
warehouses = [warehouse.as_dict() for warehouse in warehouses if warehouse.name == f"{customer_name} SQL Warehouse 2XL 1-40"]
len(warehouses)

if len(warehouses) > 0:
  warehouse_id = warehouses[0]['id']
else:
  warehouse_id = w.warehouses.create(
      auto_stop_mins = 10
      ,channel = Channel(name = ChannelName("CHANNEL_NAME_CURRENT"))
      ,cluster_size = "2X-Large"
      ,enable_photon = True
      ,enable_serverless_compute = True
      ,max_num_clusters = 40
      ,min_num_clusters = 1
      ,name = f"{customer_name} SQL Warehouse 2XL 1-40"
  ).id

warehouse_id

In [0]:
w.warehouses.update_permissions(
    warehouse_id=warehouse_id,
    access_control_list=[
        WarehouseAccessControlRequest(
            group_name='users',
            permission_level=WarehousePermissionLevel("CAN_MONITOR")
        )
    ]
)